This notebook is used to create the input file for the plinder_stratify program, which is part of the [PLINDER](https://github.com/plinder-org/plinder) software and database.  It reads two files, one containing the pdb ids for the training set and another containing the pdb ids for the test set.  

In [5]:
import pandas as pd

Make sure we can see all the columns in the PLINDER dataframe

In [6]:
pd.options.display.max_columns = None

Point to the PLINDER installation

In [7]:
PLINDER_DIR = "/home/pwalters/DATA2/PLINDER/"

Read the PLINDER index

In [8]:
index_df = pd.read_parquet(f"{PLINDER_DIR}/index/annotation_table_nonredundant.parquet")

In [10]:
index_df.head()

,entry_pdb_id,entry_release_date,entry_oligomeric_state,entry_determination_method,entry_keywords,entry_pH,entry_resolution,entry_rfree,entry_r,entry_clashscore,entry_percent_rama_outliers,entry_percent_rota_outliers,entry_data_completeness,entry_percent_RSRZ_outliers,entry_atom_count,entry_molprobity,entry_mean_b_factor,entry_median_b_factor,entry_pdbx_resolution,entry_pdbx_reflns_resolution,entry_meanI_over_sigI_obs,entry_r_minus_rfree,entry_pass_validation_criteria,system_id,system_pdb_id,system_biounit_id,system_type,system_num_pocket_residues,system_num_interactions,system_num_interacting_protein_chains,system_num_neighboring_protein_chains,system_num_ligand_chains,system_has_kinase_inhibitor,system_pocket_CATH,system_pocket_SCOP2B,system_pocket_Pfam,system_pocket_UniProt,system_pocket_ECOD,system_pocket_ECOD_t_name,suitable_for_ml_training,system_ligand_num_residues,system_ligand_num_processed_residues,system_ligand_percent_processed_residues,system_ligand_average_rsr,system_ligand_average_rsrz,system_ligand_average_rscc,system_ligand_average_occupancy,system_ligand_percent_rsr_under_threshold,system_ligand_percent_rscc_over_threshold,system_ligand_percent_occupancy_over_threshold,system_ligand_average_b_factor,system_ligand_unknown_residue_count,system_ligand_atom_count,system_ligand_heavy_atom_count,system_ligand_num_unresolved_heavy_atoms,system_ligand_max_alt_count,system_ligand_percent_outliers_geometry,system_ligand_percent_outliers_density,system_ligand_percent_outliers_chirality,system_ligand_percent_outliers_clashes,system_pocket_num_residues,system_pocket_num_processed_residues,system_pocket_percent_processed_residues,system_pocket_average_rsr,system_pocket_average_rsrz,system_pocket_average_rscc,system_pocket_average_occupancy,system_pocket_percent_rsr_under_threshold,system_pocket_percent_rscc_over_threshold,system_pocket_percent_occupancy_over_threshold,system_pocket_average_b_factor,system_pocket_unknown_residue_count,system_pocket_atom_count,system_pocket_heavy_atom_count,system_pocket_num_unresolved_heavy_atoms,system_pocket_max_alt_count,system_pocket_percent_outliers_geometry,system_pocket_percent_outliers_density,system_pocket_percent_outliers_chirality,system_pocket_percent_outliers_clashes,system_pass_validation_criteria,system_interacting_protein_chains,system_interacting_protein_chains_auth_id,system_interacting_protein_chains_entity_id,system_interacting_protein_chains_length,system_interacting_protein_chains_CATH,system_interacting_protein_chains_SCOP2B,system_interacting_protein_chains_Pfam,system_interacting_protein_chains_UniProt,system_interacting_protein_chains_ECOD,system_interacting_protein_chains_num_residues,system_interacting_protein_chains_num_processed_residues,system_interacting_protein_chains_percent_processed_residues,system_interacting_protein_chains_average_rsr,system_interacting_protein_chains_average_rsrz,system_interacting_protein_chains_average_rscc,system_interacting_protein_chains_average_occupancy,system_interacting_protein_chains_percent_rsr_under_threshold,system_interacting_protein_chains_percent_rscc_over_threshold,system_interacting_protein_chains_percent_occupancy_over_threshold,system_interacting_protein_chains_average_b_factor,system_interacting_protein_chains_unknown_residue_count,system_interacting_protein_chains_atom_count,system_interacting_protein_chains_heavy_atom_count,system_interacting_protein_chains_num_unresolved_heavy_atoms,system_interacting_protein_chains_max_alt_count,system_interacting_protein_chains_percent_outliers_geometry,system_interacting_protein_chains_percent_outliers_density,system_interacting_protein_chains_percent_outliers_chirality,system_interacting_protein_chains_percent_outliers_clashes,system_neighboring_protein_chains,system_neighboring_protein_chains_auth_id,system_neighboring_protein_chains_entity_id,system_neighboring_protein_chains_length,system_neighboring_protein_chains_CATH,system_neighboring_protein_chains_SCOP2B,system_neig

Read the pdb ids for the training set. 

In [12]:
train_df = pd.read_csv("train.csv")
train_df.head()

,pdb_id
0,4rek
1,2wfj
2,3x34
3,5yce
4,7a5m


Read the pdb ids for the test set. 

In [13]:
test_df = pd.read_csv("test.csv")
test_df.head()

,pdb_id
0,7kc5
1,7n03
2,7wux
3,5sb2
4,7lt0


To run plinder_stratify, we need to create a file that contains the PLINDER system_ids for the training and test sets, along with "train" or "test" labels.
```
system_id,split
4rek__1__1.A__1.B,train
2wfj__1__1.A__1.B,train
3x34__1__1.A__1.B,train
5yce__1__1.A__1.B,train
7a5m__1__1.A__1.C,train
7b94__2__1.B__1.I,test
7jhq__1__2.D__2.Q,test
7jhq__2__1.C__1.L,test
7jhq__2__2.B__2.J,test
7akl__1__1.A__1.B,test
```

Get the PLINDER system_id for the training set. 

In [56]:
train_system_id_df = index_df.query("entry_pdb_id in @train_df.pdb_id")[["system_id"]].copy()

Get the PLINDER system_id for the test set. 

In [57]:
test_system_id_df = index_df.query("entry_pdb_id in @test_df.pdb_id")[["system_id"]].copy()

In [60]:
len(train_system_id_df),len(test_system_id_df)

(159487, 288)

Label the training and test sets

In [41]:
train_system_id_df["split"] = "train"
test_system_id_df["split"] = "test"

Write the input for **plinder_stratify**

In [50]:
pd.concat([train_system_id_df.reset_index(),test_system_id_df.reset_index()]).drop("index",axis=1).to_csv("train_test.csv",index=False)